In [ ]:
def find_ind_data(ind, path):
    filename = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and ind+".nii.gz"  in f]
    return filename

In [ ]:
def get_data(sessionpath, filename_data):
    """
    Gets the raw data together with MRID segmentation and anatomical segmentation
    filename_data: filename of the raw T2*Map MGE data
    img_slice: image slice of interest
    """
    anatpath = os.path.join(sessionpath, "anat")
    filename_data_full = ".".join((filename_data,"nii", "gz"))
    filename_segmentation=".".join((filename_data+"-segmentation", "nii", "gz"))
    filename_anat=".".join((filename_data+"-anat", "nii", "gz"))

    nii_data, data=read_data(os.path.join(anatpath, filename_data_full))
    _, segmentation=read_data(os.path.join(anatpath, filename_segmentation))
    _, anat=read_data(os.path.join(anatpath, filename_anat))

    labelsdf=read_labels(os.path.join(sessionpath, "anat", "labels.txt"))
    print(labelsdf)
    
    print("Data shape of anatomy segmentation" + str(np.shape(anat)))
    print("Data shape of MRI data" + str(np.shape(data)))
    print("Data shape of MRID segmentation" + str(np.shape(segmentation)))

    # print("Voxel dimensions: " + str(nii_data.header['pixdim']))
    return nii_data, data, segmentation, anat, labelsdf

In [ ]:
def get_sessions(directory):
    sessions=[]
    for subdir in os.listdir(directory):
        if "ses-" in subdir:
            sessions.append(subdir)
    return sessions

def get_npy_files(directory):
    npy_files=[]
    for subdir in os.listdir(directory):
        if ".npy" in subdir:
            npy_files.append(subdir)
    
    return npy_files

In [2]:
def read_data(path):
    """
    Reads data from nii.gz file
    inputs;
    path: path to the file
    returns; 
    nii: nii object
    data: mri volume as np array
    """
    nii=nib.load(path)
    data=np.asanyarray(nii.dataobj)
    
    return nii, data
        
def save_nii(data, affine, path2save, header=None):
    nii2save=nib.Nifti1Image(data, affine, header=header)
    nib.save(nii2save, path2save)
    

In [3]:
def read_labels(labels_path):
    """
    Reading SimpleITK-Snap generated labels
    """
    labels=[]
    anats=[]
    with open(labels_path) as f:
        lines = f.readlines()
        for i,line in enumerate(lines):
            if i>14:
                if i==7:
                    anat= ' '.join(line.split('\\cf')[1].split()[8:])
                    anats.append(anat.split("\"")[1])
                    labels.append(int(line.split('\\cf')[1].split()[0]))
                else:
                    anat=' '.join(line.split()[7:])
                    anats.append(anat.split("\"")[1])
                    labels.append(int(line.split()[0]))

    df=pd.DataFrame()
    df["Labels"]=labels
    df["Anatomical Regions"]=anats
    return df

In [ ]:
def read_label_colormap(labels_path):
    labels=[]
    cmap_arr = np.zeros((223,4))

    with open(labels_path) as f:
        lines = f.readlines()
        for i,line in enumerate(lines):
            if i>14:
                cmap_arr[i-14,0]=int(line.split()[1])/255
                cmap_arr[i-14,1]=int(line.split()[2])/255
                cmap_arr[i-14,2]=int(line.split()[3])/255
                cmap_arr[i-14,3]=1

    new_cmap = ListedColormap(cmap_arr)
    
    return new_cmap

In [ ]:
def read_whs_labels(path):
    """
    Reads the labels from WHS atlas
    """
#     labels_path='./atlas_labels.rtf'
    labels=[]
    anats=[]
    with open(path) as f:
        lines = f.readlines()
        for i,line in enumerate(lines):
            if i>6 and i%2==1:
                if i==7:
                    anat= ' '.join(line.split('\\cf')[1].split()[8:])
    #                 label=[line.split('\\cf')[1].split()[0], anat]
                    anats.append(anat.split("\"")[1])
                    labels.append(int(line.split('\\cf')[1].split()[0]))
    #                 print(label)
                else:
                    anat=' '.join(line.split()[7:])
    #                 label=[line.split()[0], anat]
                    anats.append(anat.split("\"")[1])
                    labels.append(int(line.split()[0]))
    #                 print(label)

    df=pd.DataFrame()
    df["Labels"]=labels
    df["Anatomical Regions"]=anats
    return df

In [ ]:
def read_png(path, scale=cv2.IMREAD_GRAYSCALE):
    img=cv2.imread(path, scale)

    return img

In [ ]:
def sort_directory(directory):
    items = os.listdir(directory)
    sorted_items = sorted(items)
    return sorted_items

In [ ]:
def load_imgs(path):
    files=sort_directory(path)
    for idx,file in enumerate(files):
        filename = os.fsdecode(file)
        if filename.endswith(".png"):
            img=read_png(path+filename)
            if idx==0:
                x=np.shape(img)[0]
                y=np.shape(img)[1]
                data=np.zeros((x,y,1))
                data[:,:,0]=img
            else:
                data=np.append(data,img[...,newaxis],2)
    print(np.shape(data))
    return data

In [ ]:
def read_dicom(path):
    ds = pydicom.dcmread(path)
    pixel_array=np.flip(ds.pixel_array)
    img = (np.maximum(pixel_array,0) / pixel_array.max()) * 255.0
    img = np.uint8(img)
    
    return ds, img